<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/MS_Service_Blank_Subtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import pandas as pd

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [27]:
#Blank_EtOAc = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/EtOAC.csv', error_bad_lines=False)
DCM = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/DCM_new2.csv', error_bad_lines=False)
ER_0789a_n = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0789a_n2.csv', error_bad_lines=False)
ER_0788a_n = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0788a_n2.csv', error_bad_lines=False)
ER_0788b_n = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0788b_n2.csv', error_bad_lines=False)
ER_0787a_n = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0787a_n2.csv', error_bad_lines=False)
ER_0787b_n = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0787a_n2.csv', error_bad_lines=False)
ER_0792a_n = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0792a_n2.csv', error_bad_lines=False)

<ipython-input-27-51f3e8f7a8e5>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  DCM = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/DCM_new2.csv', error_bad_lines=False)
<ipython-input-27-51f3e8f7a8e5>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ER_0789a_n = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0789a_n2.csv', error_bad_lines=False)
<ipython-input-27-51f3e8f7a8e5>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ER_0788a_n = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0788a_n2.csv', error_bad_lines=False)
<ipython-input-27-51f3e8f7a8e5>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future ver

In [28]:
#Blank_EtOAc.fillna(0, inplace=True)
DCM.fillna(0, inplace=True)
ER_0789a_n.fillna(0, inplace=True)
ER_0788a_n.fillna(0, inplace=True)
ER_0788b_n.fillna(0, inplace=True)
ER_0787a_n.fillna(0, inplace=True)
ER_0787b_n.fillna(0, inplace=True)
ER_0792a_n.fillna(0, inplace=True)

In [29]:
#print(Blank_EtOAc.shape)
print(DCM.shape)
print(ER_0789a_n.shape)
print(ER_0788a_n.shape)
print(ER_0788b_n.shape)
print(ER_0787a_n.shape)
print(ER_0787b_n.shape)
print(ER_0792a_n.shape)

(74, 21)
(210, 21)
(208, 21)
(223, 21)
(122, 21)
(122, 21)
(203, 21)


In [30]:
def combine_df(df1, df2):
    combined_df = pd.concat([df1, df2], ignore_index=True)
    return combined_df

def exclusion_list(sample_df, blank_df, retention_time_threshold=1):
    sample_df_copy = sample_df.copy()
    blank_df_copy = blank_df.copy()

    exclusion_list = []

    for index_sample, row_sample in sample_df_copy.iterrows():
        for index_blank, row_blank in blank_df_copy.iterrows():
            if row_sample["Formula (mol ion)"] == row_blank["Formula (mol ion)"]:
                if abs(row_sample["Retention Time"] - row_blank["Retention Time"]) > retention_time_threshold:
                    exclusion_list.append(index_sample)
                    break

    return exclusion_list

def remove_rows(df, exclusion_list):
    df_copy = df.copy()

    df_copy = df_copy.drop(exclusion_list)

    return df_copy

def filter_df(df):
    rows_to_remove = []

    for index, row in df.iterrows():
        if row['HRF Score'] < 85 or row['SI'] < 600:
            rows_to_remove.append(index)

    filtered_df = df.drop(rows_to_remove)

    return filtered_df

def rm_no_RI(df):
    rows_to_remove = []

    for index, row in df.iterrows():
        if row['Calculated RI'] == 0:
            rows_to_remove.append(index)

    filtered_df = df.drop(rows_to_remove)

    return filtered_df

In [31]:
#Blank_data_df = combine_df(Blank_EtOAc, Blank_DCM)

ER_0789a_n_el = exclusion_list(ER_0789a_n, DCM)
ER_0789a_n_rr= remove_rows(ER_0789a_n, ER_0789a_n_el)

ER_0788a_n_el = exclusion_list(ER_0788a_n, DCM)
ER_0788a_n_rr = remove_rows(ER_0788a_n, ER_0788a_n_el)

ER_0788b_n_el = exclusion_list(ER_0788b_n, DCM)
ER_0788b_n_rr = remove_rows(ER_0788b_n, ER_0788b_n_el)

ER_0787a_n_el = exclusion_list(ER_0787a_n, DCM)
ER_0787a_n_rr = remove_rows(ER_0787a_n, ER_0787a_n_el)

ER_0787b_n_el = exclusion_list(ER_0787b_n, DCM)
ER_0787b_n_rr = remove_rows(ER_0787b_n, ER_0787b_n_el)

ER_0792a_n_el = exclusion_list(ER_0792a_n, DCM)
ER_0792a_n_rr = remove_rows(ER_0792a_n, ER_0792a_n_el)

In [32]:
print(ER_0789a_n_rr.shape)
print(ER_0788a_n_rr.shape)
print(ER_0788b_n_rr.shape)
print(ER_0787a_n_rr.shape)
print(ER_0787b_n_rr.shape)
print(ER_0792a_n_rr.shape)

(84, 21)
(81, 21)
(87, 21)
(64, 21)
(64, 21)
(107, 21)


In [33]:
ER_0789a_n_filtered = filter_df(ER_0789a_n_rr)
ER_0788a_n_filtered = filter_df(ER_0788a_n_rr)
ER_0788b_n_filtered = filter_df(ER_0788b_n_rr)
ER_0787a_n_filtered = filter_df(ER_0787a_n_rr)
ER_0787b_n_filtered = filter_df(ER_0787b_n_rr)
ER_0792a_n_filtered = filter_df(ER_0792a_n_rr)

In [34]:
print(ER_0789a_n_filtered.shape)
print(ER_0788a_n_filtered.shape)
print(ER_0788b_n_filtered.shape)
print(ER_0787a_n_filtered.shape)
print(ER_0787b_n_filtered.shape)
print(ER_0792a_n_filtered.shape)

(43, 21)
(46, 21)
(49, 21)
(41, 21)
(41, 21)
(68, 21)


In [35]:
ER_0789a_n_filtered.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0789a_n2_filtered.csv')
ER_0788a_n_filtered.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0788a_n2_filtered.csv')
ER_0788b_n_filtered.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0788b_n2_filtered.csv')
ER_0787a_n_filtered.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0787a_n2_filtered.csv')
ER_0787b_n_filtered.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0787b_n2_filtered.csv')
ER_0792a_n_filtered.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0792a_n2_filtered.csv')

In [36]:
ER_0789a_n_RIR = rm_no_RI(ER_0789a_n_filtered)
ER_0788a_n_RIR = rm_no_RI(ER_0788a_n_filtered)
ER_0788b_n_RIR = rm_no_RI(ER_0788b_n_filtered)
ER_0787a_n_RIR = rm_no_RI(ER_0787a_n_filtered)
ER_0787b_n_RIR = rm_no_RI(ER_0787b_n_filtered)
ER_0792a_n_RIR = rm_no_RI(ER_0792a_n_filtered)

In [37]:
print(ER_0789a_n_RIR.shape)
print(ER_0788a_n_RIR.shape)
print(ER_0788b_n_RIR.shape)
print(ER_0787a_n_RIR.shape)
print(ER_0787b_n_RIR.shape)
print(ER_0792a_n_RIR.shape)

(38, 21)
(36, 21)
(35, 21)
(33, 21)
(33, 21)
(57, 21)


In [38]:
ER_0789a_n_RIR.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0789a_n2_RIR.csv')
ER_0788a_n_RIR.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0788a_n2_RIR.csv')
ER_0788b_n_RIR.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0788b_n2_RIR.csv')
ER_0787a_n_RIR.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0787a_n2_RIR.csv')
ER_0787b_n_RIR.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0787b_n2_RIR.csv')
ER_0792a_n_RIR.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0792a_n2_RIR.csv')